In [1]:
import torch
import numpy as np
from pytorch3d.structures import Pointclouds
import pytorch3d.renderer as rdr
import matplotlib.pyplot as plt
import imageio
from tqdm.notebook import tqdm

In [2]:
def get_points_renderer(
    image_size=512, 
    device=None,
    radius=0.01,
    background_color=(1, 1, 1)
):
    raster_settings = rdr.PointsRasterizationSettings(
        image_size=image_size,
        radius=radius,
        points_per_pixel=10
    )
    
    renderer = rdr.PointsRenderer(
        rasterizer= rdr.PointsRasterizer(
            cameras= None,
            raster_settings= raster_settings
        ),
        compositor= rdr.AlphaCompositor()
    )
    
    return renderer

In [3]:
def create_point_cloud(rgb, depth, mask, camera, device):
    points, colors = unproject_depth_image(
        rgb,
        depth,
        mask,
        camera
    )
    
    point_cloud = Pointclouds(
        points=[points.to(device)],
        features=[colors.to(device)]
    )
    
    return point_cloud

In [4]:
def render_point_clouds(point_cloud, device, num_views=30, distance=6.0):
    renderer = get_points_renderer(
        image_size=512,
        device=device
    )
    
    images = []
    angles = torch.linspace(-180, 180, num_views)
    
    for angle in tqdm(angles):
        R, T = rdr.look_at_view_transform(
            dist=distance,
            elev=30.0,
            azim=angle.item()
        )
        cameras = rdr.FoVOrthographicCameras(
            R=R,
            T=T,
            device=device
        )
        
        rend = renderer(point_cloud, cameras=cameras)
        image = rend[0, ..., :3].cpu().numpy()
        images.append((image * 255).astype(np.uint8))
    
    return images

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_dict = load_rgbd_data()

pc1 = create_point_cloud(
    data_dict["rgb1"],
    data_dict["depth1"],
    data_dict["mask1"],
    data_dict["camera1"],
    device
)

# Create second point cloud
pc2 = create_point_cloud(
    data_dict["rgb2"],
    data_dict["depth2"],
    data_dict["mask2"],
    data_dict["camera2"],
    device
)

NameError: name 'load_rgbd_data' is not defined